In [8]:
import os

In [9]:
%pwd

'd:\\Machine Learning\\VisaEligibilityPredictor-AU'

In [10]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class ModelEvalConfig:
      root_dir: Path
      x_val_file_path: Path
      y_val_file_path: Path
      model: Path
      metric: Path

In [11]:
from src.visaPrediction.utils import *
from src.visaPrediction.constants import *


class ConfigurationManager:
      def __init__(
            self,
            config = CONFIG_FILE_PATH,
            schema = SCHEMA_FILE_PATH,
            params = PARAMS_FILE_PATH
      ):
            self.config = load_yaml_file(config)
            self.schema = load_yaml_file(schema)
            self.params = load_yaml_file(params)

            create_directory([self.config.artifact_root])
      

      def get_model_eval_config(self) -> ModelEvalConfig:
            config = self.config.model_evaluation

            create_directory([config.root_dir])

            model_eval_config = ModelEvalConfig(
                  root_dir=config.root_dir,
                  x_val_file_path=config.x_val_file,
                  y_val_file_path=config.y_val_file,
                  model=config.model,
                  metric=config.metric_file
            )

            return model_eval_config

In [12]:
import os
import joblib
from pathlib import Path
from box.config_box import ConfigBox
from src.visaPrediction import logger
from src.visaPrediction.utils import *
from sklearn.metrics import classification_report

class ModelEval:
      def __init__(self, config: ModelEvalConfig):
            self.config = config

      
      def eval_mertics(self, actual, pred) -> ConfigBox:
            data = classification_report(actual, pred, output_dict=True)
            return ConfigBox(data)
      
      def val_data(self) -> None:
            x_val = load_file(Path(self.config.x_val_file_path))
            y_val = load_file(Path(self.config.y_val_file_path))
            model = joblib.load(self.config.model)

            y_pred = model.predict(x_val)

            data = self.eval_mertics(y_val, y_pred)

            Model_Report = {
                  'Class_0': {
                        'precision': data['0']['precision'],
                        'recall': data['0']['recall'],
                        'f1-score': data['0']['f1-score'],
                        'support': data['0']['support']
                  },
                  'Class_1': {
                        'precision': data['1']['precision'],
                        'recall': data['1']['recall'],
                        'f1-score': data['1']['f1-score'],
                        'support': data['1']['support']
                  },
                  'accuracy': data['accuracy'],
            }
            save_json(path=Path(self.config.metric), data = Model_Report)
            logger.info(f"Model evaluation successfully and report saved on: {self.config.metric}")

In [13]:
try:
      config = ConfigurationManager()
      model_eval_config = config.get_model_eval_config()
      model_eval = ModelEval(config=model_eval_config)
      model_eval.val_data()
except Exception as e:
      raise e

[2026-02-15 11:08:43,874: INFO: __init__: yaml file: config\config.yaml loaded successfully]
[2026-02-15 11:08:43,878: INFO: __init__: yaml file: schema.yaml loaded successfully]
[2026-02-15 11:08:43,882: INFO: __init__: yaml file: params.yaml loaded successfully]
[2026-02-15 11:08:43,884: INFO: __init__: File directory: artifact created successfully]
[2026-02-15 11:08:43,886: INFO: __init__: File directory: artifact/model_evaluation created successfully]


[2026-02-15 11:08:47,177: INFO: __init__: json file: artifact\model_evaluation\mertic.json saved successfully]
[2026-02-15 11:08:47,177: INFO: 757618444: Model evaluation successfully and report saved on: artifact/model_evaluation/mertic.json]
